<a href="https://colab.research.google.com/github/Chraebe/SummarizationCourse/blob/main/RDF_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RDF Summarization and Versioning

In [ ]:
 !pip install rdflib